## Szintetikus regiszteradat-generálás  
*(English version below)*

Ez a notebook a repozitórium `README.md` fájljában hivatkozott szakdolgozat **„Szintetikus regiszteradat generálása”** című, **„A”** jelű mellékletében rögzített adatgenerálási specifikáció reprodukálható \textsf{R} megvalósítását mutatja be.

A generálás eredménye egy esetszintű, regiszterjellegű szintetikus adatállomány, ahol ugyanabban a táblában egyszerre szerepelnek  
(i) a megfigyelt, cenzorálást is tartalmazó regisztermezők (`F`, `delta`), valamint  
(ii) a cenzorálatlan eseményfolyamatot rögzítő referencia-mezők (`F_true`, `delta_true`).

### Kapcsolódás a specifikációhoz

Az adatgeneráló eljárás teljes implementációja a `R/notebooks/synthetic_registry_yearly.R` fájlban található. A notebook szerepe a script paraméterezett futtatása, majd az így előállított adat felhasználása a kiterjesztett (több indexdátumra értelmezett) Crouch-féle prevalenciabecslő eljárás becslési pontosságának vizsgálatához.

### Implementált komponensek (a specifikáció szerint)

1. **Időtengely és regiszterablak:** `t_orig` a választott origó dátum, `t_min` és `t_max` az incidenciagenerálás kezdő és záró dátuma, míg `t_end` a megfigyelési horizont vége.
2. **Éves incidenciafolyamat és diagnózisnapok:**
   - naptári évek szerinti bontás,
   - determinisztikus trend az éves várható intenzitásra  
     (`lambda_bar_y = lambda0 * (1+gamma)^(y-Y_min)`),
   - multiplikatív lognormális bizonytalanság (`U_y`),
   - éves esetszámok Poisson-modellből (`N_y`),
   - éven belüli diagnózisnapok diszkrét egyenletes mintavétele.
3. **Kovariánsok generálása:** hierarchikus séma (`X1`, `X2`, `X3`) Bernoulli/Beta/Normal komponensekkel.
4. **Latens eseményidő:** kovariánsfüggő generalizált gamma modellből (`T`; paraméterek: `gg_lam`, `gg_k`, `gg_b`, `gg_q`), nap pontosságú diszkretizálással.
5. **Referencia-mezők képzése:** `F_true`, `delta_true`.
6. **Megfigyelési mechanizmus:**
   - adminisztratív cenzorálás,
   - véletlen LTFU-cenzorálás,
   - megfigyelt mezők képzése (`F`, `delta`).
7. **Végső esetszintű tábla:** egységes, emberi olvashatóságot támogató oszlopnév-konvencióval.

### Notebook felépítése

1. A kezdő kódcella betölti a szükséges környezetet, majd importálja az adatgeneráló scriptet  
   (`source("../R/synthetic_registry_yearly.R")`).
2. A következő kódcella egy rögzített paraméterezés mellett legenerálja a szintetikus adatállományt (`synthetic_registry`).
3. A záró kódcella egy megadott indexdátumra prevalenciaszámításokat végez külön
   a referencia-mezők, illetve külön a megfigyelt (cenzorált) mezők alapján.

---
*(English version)*
## Synthetic Registry Data Generation  

This notebook presents a reproducible **R** implementation of the data-generating specification documented in Appendix **“A”**, titled **“Synthetic Registry Data Generation”**, of the thesis referenced in the repository’s `README.md` file.

The output of the generation procedure is an individual-level, registry-style synthetic dataset in which the same table simultaneously contains  
(i) the observed registry fields including censoring (`F`, `delta`), and  
(ii) the reference fields describing the uncensored event process (`F_true`, `delta_true`).

### Relation to the Specification

The complete implementation of the data-generating procedure is provided in the file `R/notebooks/synthetic_registry_yearly.R`.  
The role of this notebook is to execute the script with a fixed parameterization and subsequently use the generated dataset to evaluate the estimation accuracy of the extended (multi-index-date) Crouch-type prevalence estimator.

### Implemented Components (According to the Specification)

1. **Time axis and registry window:** `t_orig` denotes the chosen origin date; `t_min` and `t_max` define the start and end of incidence generation, while `t_end` marks the end of the observation horizon.  

2. **Annual incidence process and diagnosis dates:**
   - calendar-year decomposition,  
   - deterministic trend in the expected annual intensity  
     (`lambda_bar_y = lambda0 * (1+gamma)^(y - Y_min)`),  
   - multiplicative lognormal uncertainty (`U_y`),  
   - annual case counts from a Poisson model (`N_y`),  
   - within-year diagnosis dates sampled from a discrete uniform distribution.  

3. **Covariate generation:** hierarchical scheme (`X1`, `X2`, `X3`) using Bernoulli, Beta, and Normal components.  

4. **Latent event time:** covariate-dependent generalized gamma model (`T`; parameters: `gg_lam`, `gg_k`, `gg_b`, `gg_q`), discretized to daily resolution.  

5. **Construction of reference fields:** `F_true`, `delta_true`.  

6. **Observation mechanism:**
   - administrative censoring,  
   - random LTFU censoring,  
   - construction of observed fields (`F`, `delta`).  

7. **Final individual-level table:** unified column naming convention supporting human readability.  

### Structure of the Notebook

1. The first code cell loads the required environment and imports the data-generating script  
   (`source("../R/synthetic_registry_yearly.R")`).  

2. The next code cell generates the synthetic dataset (`synthetic_registry`) under a fixed parameterization.  

3. The final code cell performs prevalence calculations at a specified index date separately  
   based on the reference fields and on the observed (censored) fields.


In [31]:
options(stringsAsFactors = FALSE)

project_root <- if (file.exists("DESCRIPTION")) {
  "."
} else if (file.exists("../DESCRIPTION")) {
  ".."
} else {
  stop("Could not find project root (DESCRIPTION).")
}

if (!requireNamespace("devtools", quietly = TRUE)) {
  stop("Please install devtools to load the local package.")
}

suppressPackageStartupMessages(devtools::load_all(project_root, quiet = TRUE))

loaded_path <- normalizePath(getNamespaceInfo("rprev", "path"), winslash = "/")
expected_path <- normalizePath(project_root, winslash = "/")
if (loaded_path != expected_path) {
  stop("Loaded rprev is not the local repo version: ", loaded_path)
}
cat("Loaded local fork (package name: rprev) from: ", loaded_path, "\n", sep = "")

source(file.path(project_root, "notebooks", "synthetic_registry_yearly.R"))


Loaded local fork (package name: rprev) from: C:/Users/600972868/OneDrive - BT Plc/Desktop/Sajat Mappa/00 - BGE/Alkalmazott Matematika/Szakdolgozat/repo/rprev-ext


In [32]:
# --- Git meta (branch + commit) kiirasa ---
git_cmd <- function(cmd) {
  out <- tryCatch(
    system(cmd, intern = TRUE, ignore.stderr = TRUE),
    error = function(e) character(0)
  )
  if (length(out) == 0) NA_character_ else out[[1]]
}

if (!exists("project_root", inherits = FALSE)) {
  project_root <- if (file.exists("DESCRIPTION")) {
    "."
  } else if (file.exists("../DESCRIPTION")) {
    ".."
  } else {
    stop("Could not find project root (DESCRIPTION).")
  }
}

old_wd <- getwd()
on.exit(setwd(old_wd), add = TRUE)
setwd(project_root)

branch <- git_cmd("git rev-parse --abbrev-ref HEAD")
commit <- git_cmd("git rev-parse HEAD")

cat("Git branch: ", branch, "\n", sep = "")
cat("Git commit: ", commit, "\n", sep = "")

if (is.na(branch) || is.na(commit)) {
  warning("Could not read git branch/commit (is git installed, and is this a git repo?).")
}


Git branch: master
Git commit: 7a6e47e89c327c86c07ba68434fb4132adbc3539


In [33]:
params <- list(
  t_orig = "1990-01-01",
  t_min = "2000-01-01",
  t_max = "2025-01-01",
  t_end = "2026-12-31",

  # Yearly incidence trend parameters
  inc_lam0 = 850,
  inc_g = 0.04,
  inc_sdlog = 0.20,
  strict_calendar_bounds = TRUE,

  # Covariates
  p_x1 = 0.42,
  beta_a_x1 = c(2.5, 5.5),
  beta_b_x1 = c(5.0, 2.0),
  x2_range = c(20, 85),
  x3_mu = c(-0.2, 0.5, 0.03),
  x3_sd = 1.1,

  # Latent survival model
  gg_lam = 2200,
  gg_k = 1.3,
  gg_b = c(-0.4, 0.012, 0.7),
  gg_q = 1.0,

  # Censoring
  p_ltfu = 0.25,

  # Reproducibility
  seed = 20260213
)

synthetic_registry <- do.call(generate_synthetic_registry_yearly, params)

cat("Generated rows:", nrow(synthetic_registry), "\n")
head(synthetic_registry, 5)


Generated rows: 35322 


,case_id_id,diagnosis_date_D,covariate_binary_x1,covariate_scaled_x2,covariate_hidden_x3,last_followup_date_F,event_observed_delta,true_event_date_F_true,true_event_indicator_delta_true
,<int>,<date>,<int>,<dbl>,<dbl>,<date>,<int>,<date>,<int>
1,1,2016-05-25,0,37.77020,-1.2074446,2020-03-23,1,2020-03-23,1
2,2,2012-05-15,0,39.65570,0.9807039,2013-08-10,1,2013-08-10,1
3,3,2021-05-08,1,59.72376,0.2926514,2026-12-31,0,2028-06-10,1
4,4,2021-11-12,0,47.84596,0.3209795,2022-09-07,1,2022-09-07,1
5,5,2015-03-24,1,63.84817,2.7869120,2015-12-24,1,2015-12-24,1


In [35]:
index_date <- as.Date(c(
  "2000-01-01",
  "2010-01-01",
  "2015-01-01",
  "2014-01-01",
  "2015-01-01",
  "2016-01-01",
  "2017-01-01",
  "2018-01-01"
))

population_size <- 10e3

prevalence_summary <- count_prevalence_at_index(
  synthetic_registry,
  index_date = index_date,
  population_size = population_size,
  include_index_day = FALSE
)

print(prevalence_summary, row.names = FALSE)


 index_date followup_basis prevalent_cases diagnosed_cases prevalence_rate
 2000-01-01         true_F               1               1          0.0001
 2000-01-01     observed_F               1               1          0.0001
 2010-01-01         true_F            2582           10585          0.2582
 2010-01-01     observed_F            2282           10585          0.2282
 2015-01-01         true_F            2765           16813          0.2765
 2015-01-01     observed_F            2424           16813          0.2424
 2014-01-01         true_F            2986           15670          0.2986
 2014-01-01     observed_F            2633           15670          0.2633
 2015-01-01         true_F            2765           16813          0.2765
 2015-01-01     observed_F            2424           16813          0.2424
 2016-01-01         true_F            3027           18482          0.3027
 2016-01-01     observed_F            2660           18482          0.2660
 2017-01-01         true_